In [11]:
using Convex
#using Gurobi
constraints = []

0-element Array{Any,1}

In [12]:
# Problem definition
nctrl = 10; # number of control points
nt = 5; # number of time steps
r = 1; # radius of the motor
circ = 2*pi*r; # maximum circumference of the motor
circProfile = 2*pi*r*ones(nt);
regrate = 0.05;

In [30]:
#@variable(m, 0 <= ctrlRadius[1:nctrl,1:nt] <= r)
#@variable(m, 0 <= ctrlTheta[1:nctrl,1:nt] <= 2*pi)
# JuMP does not support complex numbers
x    = Variable(nctrl, nt)
y    = Variable(nctrl, nt)
nx   = Variable(nctrl, nt)
ny   = Variable(nctrl, nt)
l    = Variable(nctrl, nt, Positive())
circ = Variable(nt, Positive())
# for i in 1:nctrl
#     for j in 1:nt
#         constraints += x[i,j]^2 + y[i,j]^2 <= r^2
#     end
# end


LoadError: MethodError: no method matching +(::Array{Any,1}, ::Convex.LtConstraint)[0m
Closest candidates are:
  +(::Any, ::Any, [1m[31m::Any[0m, [1m[31m::Any...[0m) at operators.jl:138
  +(::Array{T,N}, [1m[31m::SparseMatrixCSC{Tv,Ti<:Integer}[0m) at sparse\sparsematrix.jl:1711
  +{T<:Convex.Constraint}([1m[31m::Array{T<:Convex.Constraint,N}[0m, ::Convex.Constraint) at C:\Users\Berk\.julia\v0.5\Convex\src\constraints/constraints.jl:175
  ...[0m

In [32]:
for i in range(1,nctrl)
    for j in 1:nt
        # Geometry bounding
        constraints += x[i,j]^2 + y[i,j]^2 <= r^2
    end
end

LoadError: MethodError: no method matching +(::Array{Any,1}, ::Convex.LtConstraint)[0m
Closest candidates are:
  +(::Any, ::Any, [1m[31m::Any[0m, [1m[31m::Any...[0m) at operators.jl:138
  +(::Array{T,N}, [1m[31m::SparseMatrixCSC{Tv,Ti<:Integer}[0m) at sparse\sparsematrix.jl:1711
  +{T<:Convex.Constraint}([1m[31m::Array{T<:Convex.Constraint,N}[0m, ::Convex.Constraint) at C:\Users\Berk\.julia\v0.5\Convex\src\constraints/constraints.jl:175
  ...[0m

In [4]:
for i in range(1,nctrl)
    for j in 1:nt
        # Geometry bounding
        constraints += [x[i,j]^2 + y[i,j]^2 <= r^2]
    end
end

# Normal vector and length (of LHS edge of current point) calculation
for j in 1:nt
    constraints +=[ nx[1,j] == -(y[1,j]-y[nctrl,j])/l[nctrl,j])
    @NLconstraint(m, ny[1,j] == (x[1,j]-x[nctrl,j])/l[nctrl,j])
    @NLconstraint(m, l[1,j]^2 == (x[1,j] - x[nctrl, j])^2 + (y[1,j] - y[nctrl, j])^2)]
    for i in 2:nctrl
        @NLconstraint(m, nx[i,j] == -(y[i,j]-y[i-1,j])/l[i-1,j])
        @NLconstraint(m, ny[i,j] == (x[i,j]-x[i-1,j])/l[i-1,j])
    end
end

# Motion of the flame front
for j in 2:nt
    constraints += [x[nctrl,j] == x[nctrl,j-1] + 0.5*nx[nctrl,j-1]*regrate + 0.5*nx[1,j-1]*regrate,
                    y[nctrl,j] == y[nctrl,j-1] + 0.5*ny[nctrl,j-1]*regrate + 0.5*ny[1,j-1]*regrate]
    for i in 1:nctrl-1
    constraints += [x[i,j] == x[i,j-1] + nx[i,j-1]*regrate + nx[i+1,j-1]*regrate,
                    y[i,j] == y[i,j-1] + ny[i,j-1]*regrate + ny[i+1,j-1]*regrate]
    end
end
    


In [38]:
objective= sum((circProfile[i]-circ[i])^2 for i in nt)

AbstractExpr with
head: qol_elem
size: (1, 1)
sign: Convex.Positive()
vexity: Convex.ConvexVexity()


In [1]:
problem = minimize(objective)
problem.constraints = constraints

LoadError: UndefVarError: minimize not defined